In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt


# Flag to run tests and visualizations for each function 
doRunTests = True


# define the files to be used

xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/015_AgePie_20211112_1_r(1).xdf"
s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/AgePie/AgePie_A16.snirf"

# xdf_fullFile = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.xdf"
# s_file = "/Users/denismottet/Documents/GitHub/NeuArm-DataAnalysis/data/ReArm.lnk/twoTestPatientsForOXY4/C1P07_20210802_1_r.snirf"



# Translation of an xdf file to snirf format 

We have the same nirs data in two formats: xdf and snirf. 

The xdf format is a general format for storing time series data (https://github.com/sccn/xdf). 

The snirf format is a format for storing nirs data (https://github.com/fNIRS/snirf).



## Load the xdf file and return only the NIRS and Event streams

In [ ]:
import pyxdf


def get_NIRS_and_Event_streams(x_file):
    """
    Load the xdf file and returns only the NIRS and Event streams
    """
    # load only the NIRS and Event streams
    data, header = pyxdf.load_xdf(
        filename=x_file,
        select_streams=[{"type": "NIRS"}, {"type": "Event"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,
        verbose=False,
    )
    # find the nirs stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "NIRS":
            nirsStream = data[i]
            break
    # find the Event stream among the list of streams
    for i in range(len(data)):
        if data[i]["info"]["type"][0] == "Event":
            eventStream = data[i]
            break
    return nirsStream, eventStream


if doRunTests:
    # load the xdf file and get the NIRS and Event streams
    nirsStream, eventStream = get_NIRS_and_Event_streams(xdf_fullFile)
    # print the name and type of each retruned stream
    print("File: {}".format(xdf_fullFile))
    print(
        "Nirs : {}, {}".format(
            nirsStream["info"]["name"][0], nirsStream["info"]["type"][0]
        )
    )
    print(
        "Event: {}, {}".format(
            eventStream["info"]["name"][0], eventStream["info"]["type"][0]
        )
    )

##     Explore the sequence of markers in the Event stream and plot the data and markers 

In [ ]:
def explore_xdf_markers_sequence(nirsStream, eventStream):
    """
    Explore the sequence of markers in the Event stream
    """

    nirsData_xdf = nirsStream["time_series"] 
    nirsTime_xdf = nirsStream["time_stamps"] 
    eventData_xdf = eventStream["time_series"] 
    eventTime_xdf = eventStream["time_stamps"] 

    # find the set of possible events using np.unique
    events = np.unique(eventData_xdf)
    print("Found {} events with labels in {}".format(len(eventData_xdf), events))

    # find all events containing the word 111 and 100
    i111 = []
    i100 = []
    for i in range(len(eventData_xdf)):
        if "111" in eventData_xdf[i][0]:
            i111.append(i)
        if "100" in eventData_xdf[i][0]:
            i100.append(i)
    print("Found {} events 111".format(len(i111)))
    print("Found {} events 100".format(len(i100)))

    iStart = i111[0]

    print(
        "First event 111 is at index {} with label {} at {:10.3f}s".format(
            iStart,
            eventData_xdf[iStart][0],
            eventTime_xdf[iStart],
        ),
    )

    # get the labels for the NIRS data
    nirsLabels = []
    for chan in nirsStream["info"]["desc"][0]["channels"][0]["channel"]:
        label = chan["label"]
        unit = chan["unit"]
        type = chan["type"]
        nirsLabels.append({"label": label, "unit": unit, "type": type})
    print("Found {} channels: ".format(len(nirsLabels)))
    for i in range(len(nirsLabels)):
        print(
            "  {:02d}: {} ({} {})".format(
                i,
                nirsLabels[i]["label"][0][8:],  # remove the first 8 characters
                nirsLabels[i]["type"][0],
                nirsLabels[i]["unit"][0],
            )
        )

    # keep only the NIRS data channels with type NIRS and unit OD
    nirs = []
    labels = []
    for i in range(len(nirsLabels)):
        if nirsLabels[i]["type"][0] == "NIRS" and nirsLabels[i]["unit"][0] == "OD":
            nirs.append(nirsData_xdf[:, i])
            labels.append(nirsLabels[i]["label"][0][8:])


    ###########################################################################################
    # make boxplot of all channels in the NIRS data, with labels in nirsLabels
    plt.figure()
    plt.boxplot(nirs)
    plt.xticks(range(1, len(labels) + 1), labels)
    # write labels vertically
    plt.xticks(rotation=90)
    plt.title("All xdf NIRS channels")
    plt.show()

    # keep only the channels that have a median lower than 3
    nirsActiveChannels = []
    labelsActiveChannels = []
    for i in range(len(nirs)):
        if np.median(nirs[i]) < 4.5:
            nirsActiveChannels.append(nirs[i])
            labelsActiveChannels.append(labels[i])

    ###########################################################################################
    # make boxplot of all channels in the **real** NIRS data channels
    plt.figure()
    plt.boxplot(nirsActiveChannels)
    plt.xticks(range(1, len(labelsActiveChannels) + 1), labelsActiveChannels)
    # write labels vertically
    plt.xticks(rotation=90)
    plt.title("xdf NIRS channels with median < 4.5")
    plt.show()

    ###########################################################################################
    # plot the NIRS data with vertical lines at the event times
    plt.figure()

    myTime_x = nirsTime_xdf - eventTime_xdf[iStart]
    myTime_x_event = eventTime_xdf - eventTime_xdf[iStart]

    # plot nirs as a function of time
    for i in range(len(nirsActiveChannels)):
        plt.plot(myTime_x, nirsActiveChannels[i], label=labelsActiveChannels[i])

    for i in range(len(eventData_xdf)):
        e = eventData_xdf[i][0]
        t = myTime_x_event[i]
        plt.axvline(x=t, color="lightgray")
        if "100" in e:
            plt.axvline(x=t, color="green")
        elif "111" in e:
            plt.axvline(x=t, color="red")

    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    # put the legend outside the plot
    ax = plt.gca()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.title("xdf NIRS data with events")
    plt.show()

    nirsActiveChannels = np.array(nirsActiveChannels).T # channels in columns
    labelsActiveChannels = np.array(labelsActiveChannels)

    return nirsActiveChannels, labelsActiveChannels


if doRunTests:
    nirsStream, eventStream = get_NIRS_and_Event_streams(xdf_fullFile)
    nirsActiveChannels, labelsActiveChannels = explore_xdf_markers_sequence(nirsStream, eventStream)

    print("nirsActiveChannels: {}".format(nirsActiveChannels.shape))
    print("labelsActiveChannels: {}".format(labelsActiveChannels.shape))
    for i in range(len(labelsActiveChannels)):
        print("  {:02d}: {}".format(i, labelsActiveChannels[i]))

# Read the snirf file

In [ ]:
# Load the snirf file
import mne

raw = mne.io.read_raw_snirf(s_file, preload=True, verbose="CRITICAL")
raw_data = raw.get_data()
print(raw_data.shape)
# plot the raw data (to see the markers-annotations)
# raw.plot()


# get the annotations (there is no Events in this file)
annotations = raw.annotations

# print all annotations
for i in range(len(annotations.description)):
    e = annotations.description[i]
    t = annotations.onset[i]
    # print("{:2d}: {:10.3f}s = {} ".format(i, t, e))

# find the first annotation containing the word 111
iStart = -1
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        iStart = i
        break
print(
    "First event 111 is at index {} with label {} at time {}".format(
        iStart, annotations.description[iStart], annotations.onset[iStart]
    ),
)

# find all annotations containing the word 111
i111s = []
for i in range(len(annotations.description)):
    if "111" in annotations.description[i]:
        i111s.append(i)
print("All events 111 are at indexes {}".format(i111s))


# Format XDF and SNIRF markers for sequence comparison

We want a simple list of markers as time & label pairs.

In [ ]:
def format_xdf_and_snirf_makers(xdf_markers_stream, snirf_annotations):
    """
    Format the markers from the xdf and snirf files as a list of times and labels
    """

    # format the xdf_markers_stream as a list of times and labels
    xdf_markers = []
    for i in range(len(xdf_markers_stream["time_stamps"])):
        xdf_markers.append(
            [
                xdf_markers_stream["time_stamps"][i],
                xdf_markers_stream["time_series"][i][0],
            ]
        )

    # format the snirf_annotations as a list of times and labels
    snirf_markers = []
    for i in range(len(snirf_annotations.onset)):
        snirf_markers.append(
            [
                snirf_annotations.onset[i],
                snirf_annotations.description[i],
            ]
        )

    return xdf_markers, snirf_markers

def format_xdf_and_snirf_nirs_data(xdf_nirs_data, snirf_nirs_data):

    # concatenate the time_stamps and time_series in a single numpy array
    xdf_nirs = np.concatenate(
        (
            np.array(xdf_nirs_data["time_stamps"]).reshape(-1, 1),
            np.array(xdf_nirs_data["time_series"]), 
        ),
        axis=1,
    )
    # concatenate the times and data from snrifs in a single numpy array
    times = snirf_nirs_data.times.reshape(-1, 1)
    data = snirf_nirs_data.get_data().T

    snirf_nirs = np.concatenate((times, data), axis=1)

    return xdf_nirs, snirf_nirs

if doRunTests: 

    x_nirs, s_nirs = format_xdf_and_snirf_nirs_data(nirsStream, raw)

    print("xdf nirs has size {}".format(x_nirs.shape))
    print("snirf nirs has size {}".format(s_nirs.shape))

    x_markers, s_markers = format_xdf_and_snirf_makers(eventStream, annotations)

    print("Found {} xdf markers".format(len(x_markers)))
    for i in range(len(x_markers)):
        print("  {:2d}: {:10.3f}s = {} ".format(i, x_markers[i][0], x_markers[i][1]))

    print("Found {} snirf markers".format(len(s_markers)))
    for i in range(len(s_markers)):
        print("  {:2d}: {:10.3f}s = {} ".format(i, s_markers[i][0], s_markers[i][1]))


# Plot the data and markers 

In [ ]:
def plot_data_and_markers(data, markers, title=""):
    """
    Plot the data and markers
    """
    # check that the data is a numpy array with at least 2 columns
    assert isinstance(data, np.ndarray)
    assert data.shape[1] >= 2

    # check that the markers is a list of lists with 2 elements
    assert isinstance(markers, list)
    assert len(markers) > 0
    assert isinstance(markers[0], list)
    assert len(markers[0]) == 2


    plt.figure()
    # plot the data
    for i in range(1, data.shape[1]):
        plt.plot(data[:, 0], data[:, i], label="{}".format(i))
    # plot the markers
    for i in range(len(markers)):
        t = markers[i][0]
        label = markers[i][1]
        plt.axvline(x=t, color="lightgray")
        if "100" in label:
            plt.axvline(x=t, color="green")
        elif "111" in label:
            plt.axvline(x=t, color="red")

    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    # put the legend outside the plot
    ax = plt.gca()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
    # Put a legend to the right of the current axis
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.title(title)
    plt.show()

if doRunTests: 
    plot_data_and_markers(x_nirs, x_markers, title="xdf data and markers")
    plot_data_and_markers(s_nirs, s_markers, title="snirf data and markers")

In [ ]:
def plot_sequence(marker_time, marker_description, data_time, data_values):
    """
    Plot the sequence of events in the snirf file
    """
    # plot the NIRS data with vertical lines at the event times
    plt.figure()

    # for each column in the NIRS data, plot the data as a function of time
    nColumns = data_values.shape[1]
    for i in range(nColumns):
        plt.plot(data_time, data_values[:,i], label="{}".format(i))

    for i in range(len(marker_time)):
        e = marker_description[i]
        if type(e) == list:
            e = e[0]
        t = marker_time[i]
        plt.axvline(x=t, color="lightgray")
        print(e)
        if "100" in e:
            plt.axvline(x=t, color="green")
        elif "111" in e:
            plt.axvline(x=t, color="red")

    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    # put the legend outside the plot
    ax = plt.gca()
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    plt.title(" NIRS data with events")
    plt.show()


if doRunTests:
    marker_time = annotations.onset
    marker_description = annotations.description
    data_time = raw.times
    data_values = raw_data.T
    plot_sequence(marker_time, marker_description, data_time, data_values)


    nirsData_xdf = nirsStream["time_series"]
    nirsTime_xdf = nirsStream["time_stamps"]
    eventData_xdf = eventStream["time_series"]
    eventTime_xdf = eventStream["time_stamps"]

    plot_sequence(eventTime_xdf, eventData_xdf, nirsTime_xdf, nirsData_xdf)
    

# find the xdf maker sequence in the snirf markers sequence

In [ ]:
def find_index_of_subsequence_in_sequence(sequence, subsequence):
    """
    Find the index of the subsequence in the sequence
    """
    iStart = None
    for i in range(len(sequence)):
        seq_beg = i
        seq_end = i + len(subsequence)
        if seq_end <= len(sequence):
            if np.allclose(subsequence, sequence[seq_beg:seq_end], atol=0.5):
                iStart = seq_beg
                break
        else:
            print("End of sequence is out of range")
            print("seq_end = {}".format(seq_end))
            print("len(sequence) = {}".format(len(sequence)))
            break
    return iStart


def check_if_labels_are_the_same(labels1, labels2):
    """
    Check if the labels are the same in the two sequences
    """
    same_labels = True
    for j in range(len(labels1)):
        # check the last 3 characters (the number)
        label1 = labels1[j][-3:]
        label2 = labels2[j][-3:]
        if label1 != label2:
            print("labels are not the same")
            print("label1 = {}".format(label1))
            print("label2 = {}".format(label2))
            same_labels = False
            break
    return same_labels


def get_only_111_and_100_markers(marker_list):
    """
    Keep only the markers 111 and 100 in the marker list
    """
    # keep only markers 111 and 100 in the data
    marker_list_111_100 = []
    for i in range(len(marker_list)):
        label = marker_list[i][1]
        if "111" in label or "100" in label:
            marker_list_111_100.append(marker_list[i])
    marker_list_111_100 = np.array(marker_list_111_100)
    return marker_list_111_100


def find_start_time_of_xdf_markers_in_snirf(x_markers, s_markers):
    """
    Find the start time of the xdf markers sequence that is also in the snirf markers
    """
    x_sequence = None
    s_sequence = None

    # keep only markers 111 and 100 in the xdf data
    x_111_100 = get_only_111_and_100_markers(x_markers)

    x_times = x_111_100[:, 0]
    x_times = np.array([float(t) for t in x_times])
    x_sequence = np.diff(x_times)
    x_labels = x_111_100[:, 1]

    # keep only markers 111 and 100 in the snirf data
    s_111_100 = get_only_111_and_100_markers(s_markers)
    s_times = s_111_100[:, 0]
    s_times = np.array([float(t) for t in s_times])
    s_sequence = np.diff(s_times)
    s_labels = s_111_100[:, 1]

    # find the sequence of markers in the snirf data that is the closest to the xdf sequence
    iStart = find_index_of_subsequence_in_sequence(s_sequence, x_sequence)
    if iStart is not None:
        # ensure that the labels are with the same number
        same_labels = check_if_labels_are_the_same(
            x_labels, s_labels[iStart : iStart + len(x_labels)]
        )
        if not same_labels:
            print("Labels are not the same")
            iStart = None

        startTime_in_snirf = float(s_111_100[iStart, 0])

    return startTime_in_snirf, x_sequence, s_sequence


if doRunTests:
    (
        time_start_in_snirf,
        xdf_sequence_111_100,
        snirf_sequence_111_100,
    ) = find_start_time_of_xdf_markers_in_snirf(x_markers, s_markers)
    if time_start_in_snirf is None:
        print("No corresponding sequence found in snirf")
    else:
        print(
            "Corresponding sequence found at index {} of markers in snirf".format(
                iStart
            )
        )
        print("beg time = {}".format(time_start_in_snirf))

        # find the index of startTime_in_snirf in the snirf sequence
        marker_times = np.array([float(t[0]) for t in s_markers])
        iStart = np.where(marker_times == time_start_in_snirf)[0][0]
        print("iStart = {}".format(iStart))

        # print all the marker times line by line
        print("Found {} markers in the snirf file".format(len(s_markers)))
        for i in range(len(s_markers)):
            delta = 0
            if i > 0:
                delta = marker_times[i] - marker_times[i - 1]
            print(
                "{:2d}: {:10.1f} {:10.1f}s = {} ".format(
                    i,
                    delta,
                    marker_times[i],
                    s_markers[i][1],
                )
            )

        # print the snirf sequence
        print(
            "Found {} markers in the snirf sequence".format(len(snirf_sequence_111_100))
        )
        for i in range(len(snirf_sequence_111_100)):
            print(
                "{:2d}: {:10.3f}s = {} ".format(
                    i,
                    snirf_sequence_111_100[i],
                    s_markers[i][1],
                )
            )

        # print the xdf sequence
        print("Found {} markers in the xdf sequence".format(len(xdf_sequence_111_100)))
        for i in range(len(xdf_sequence_111_100)):
            print(
                "{:2d}: {:10.3f}s = {} ".format(
                    i,
                    xdf_sequence_111_100[i],
                    x_markers[i][1],
                )
            )

        ###############################################################################
        # plot the xdf and snirf sequences on the same plot

        # shift the xdf sequence to the left by nan values (to match the snirf sequence)
        iStart_in_sequence = find_index_of_subsequence_in_sequence(
            snirf_sequence_111_100, xdf_sequence_111_100
        )
        # concatenate iStart nan values at the beginning of the xdf sequence
        xdf_sequence_111_100_to_plot = np.concatenate(
            (np.full(iStart_in_sequence, np.nan), xdf_sequence_111_100)
        )

        plt.figure()
        plt.plot(snirf_sequence_111_100, "o-", label="snirf")
        plt.plot(xdf_sequence_111_100_to_plot, "o-", label="xdf")
        plt.legend()
        plt.title("xdf and snirf sequences (111 100 only in both)")
        plt.show()

    # restrict the snirf sequence to the corresponding xdf sequence
    iBeg = iStart_in_sequence
    iEnd = iStart_in_sequence + len(xdf_sequence_111_100)
    snirf_sequence_111_100 = snirf_sequence_111_100[iBeg:iEnd]
    difference = xdf_sequence_111_100 - snirf_sequence_111_100
    
    # create a figure with the plot of the difference (left panel) and a boxplot of the difference (right panel)

    # create a grid with 1 row and 2 columns of different width
    f, (a0, a1) = plt.subplots(1, 2, width_ratios=[10, 1])

    # plot the difference as dots
    a0.plot(difference * 1000, "o")
    a0.set_ylabel("Difference (xdf -snirf) in milliseconds")
    a0.set_xlabel("Index")
    a0.set_title("Difference (xdf -snirf) in milliseconds")

    # make a boxplot without box o
    a1.boxplot(difference * 1000, vert=True, widths=0.8)
    a1.set_axis_off()

    plt.show()

# restrict the snirf data to the same time as the xdf data


In [ ]:
def restrict_snirf_to_xdf_time(raw, annotations, iStart, eventTime_xdf, nirsTime_xdf):
    """
    Restrict the snirf data to the same time as the xdf data
    """
    # get the time of the first and last event in the xdf data

    duration_xdf = nirsTime_xdf[-1] - nirsTime_xdf[0]

    time_first_event_xdf = eventTime_xdf[0]
    delay_first_event_xdf = time_first_event_xdf - nirsTime_xdf[0] 
    tmin = annotations.onset[iStart] - delay_first_event_xdf
    tmax = tmin + duration_xdf 

    # restrict the snirf data to the same time as the xdf data
    raw_new = mne.io.Raw.copy(raw)
    raw_new.crop(tmin=tmin, tmax=tmax)

    return raw_new, tmin


if doRunTests:

    eventData_xdf = eventStream["time_series"]
    eventTime_xdf = eventStream["time_stamps"]
    nirsTime_xdf = nirsStream["time_stamps"]


    raw_new, time_zero = restrict_snirf_to_xdf_time(
        raw, annotations, iStart, eventTime_xdf, nirsTime_xdf
    )


    snirf_labels = raw_new.ch_names
    snirf_time = raw_new.times
    snirf_data = raw_new.get_data().T
    snirf_markers = raw_new.annotations.description
    snirf_markers_time = raw_new.annotations.onset
    print("snirf_data.shape = {}".format(snirf_data.shape))
    print("snirf_time.shape = {}".format(snirf_time.shape))
    print("snirf_markers.shape = {}".format(len(snirf_markers)))




    xdf_labels = labelsActiveChannels
    xdf_data = nirsActiveChannels
    xdf_time = np.array(nirsTime_xdf)
    xdf_markers = np.array(eventData_xdf)
    xdf_markers_time = np.array(eventTime_xdf)

    print("xdf_data.shape = {}".format(xdf_data.shape))
    print("xdf_time.shape = {}".format(xdf_time.shape))
    print("xdf_markers.shape = {}".format(xdf_markers.shape))



# Re-order the xdf channels to match the snirf channels 

In [ ]:
# reorder the labels = put 757 first, then 852 
# order is unchanged within the 757 and 852 groups
xdf_labels_reordered = []
xdf_data_reordered = []
for i in range(len(xdf_labels)):
    label = xdf_labels[i]
    if "757" in label:
        xdf_labels_reordered.append(label)
        xdf_data_reordered.append(xdf_data[:, i])
for i in range(len(xdf_labels)):
    label = xdf_labels[i]
    if "852" in label:
        xdf_labels_reordered.append(label)
        xdf_data_reordered.append(xdf_data[:, i])

for i in range(len(xdf_labels_reordered)):
    print("{:02d}: {}".format(i, xdf_labels_reordered[i]))

xdf_labels = np.array(xdf_labels_reordered)
xdf_data = np.array(xdf_data_reordered).T

# Plot the snirf and xdf data

In [ ]:
# # diff the snirf data
# snirf_data = np.diff(snirf_data, axis=0)
# xdf_data = np.diff(xdf_data, axis=0)
# # add a leading zero to the snirf data
# snirf_data = np.concatenate((np.zeros((1, snirf_data.shape[1])), snirf_data), axis=0)
# xdf_data = np.concatenate((np.zeros((1, xdf_data.shape[1])), xdf_data), axis=0)
# xdf_data = - xdf_data

# # restrict snirf to channel 9 and xdf to channel 2
# snirf_selected = 9
# xdf_selected = 2
# snirf_data = snirf_data[:, snirf_selected:snirf_selected+1]
# snirf_labels = snirf_labels[snirf_selected:snirf_selected+1]
# xdf_data = xdf_data[:, xdf_selected:xdf_selected+1]
# xdf_labels = labelsActive[xdf_selected:xdf_selected+1]


plt.figure()
# plt.plot(snirf_time, snirf_data)

# plot nirs as a function of time
for i in range(len(snirf_labels)):
    plt.plot(snirf_time, snirf_data[:, i], label=snirf_labels[i])

# add a vertical line at the time of each event
for i in range(len(snirf_markers_time)):
    e = snirf_markers[i]
    t = snirf_markers_time[i] - time_zero
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")
plt.xlabel("Time (s)")
plt.title("snirf data")
# put the legend outside the plot
ax = plt.gca()
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
# Put a legend to the right of the current axis
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.show()

# check the previous plot with mne
# raw_new.plot(duration=duration_xdf, start=0, scalings="auto")


plt.figure()

# plot nirs as a function of time
for i in range(len(xdf_labels)):
    plt.plot(xdf_time, xdf_data[:, i], label=xdf_labels[i])
# plt.plot(xdf_time, xdf_data)

# add a vertical line at the time of each event
for i in range(len(xdf_markers_time)):
    e = xdf_markers[i][0]
    t = xdf_markers_time[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")
plt.xlabel("Time (s)")
plt.title("xdf data")
# put the legend outside the plot
ax = plt.gca()
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])
# Put a legend to the right of the current axis
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.show()

# Plot ONE snirf and xdf signal on the same plot

This plot shows that the snirf and xdf data are similar, but :
- the zero (by channel) is different for the two data sets
- the amplitude (by channel) is different for the two data sets
- the sign is reversed for the two data sets

In [ ]:
import copy 

# define a common time axis
x_time = xdf_time - xdf_time[0]
s_time = snirf_time - snirf_time[0]

print("xdf_time.shape = {}".format(x_time.shape))
print("snirf_time.shape = {}".format(s_time.shape))

x_labels = xdf_labels
s_labels = np.array(snirf_labels)

print("xdf_labels.shape = {}".format(x_labels.shape ))
print("snirf_labels.shape = {}".format(s_labels.shape ))


errors = []
percent_errors = []

for ch in range(len(x_labels)):

    x_dat = copy.deepcopy(xdf_data[:, ch])
    s_dat = copy.deepcopy(snirf_data[:, ch])

    # we need to transform the data to get the same values:
    # matlab code from ARTINIS to transform the data: 
    # data.dataTimeSeries = 1./exp(log(10).* [rawvals(:, 2:2:end) rawvals(:, 1:2:end)]);%change dataTimeSeries to correct values
    # in python: 
    # x_dat = 1.0 / np.exp(np.log(10) * x_dat)
    # NOTE : this is equivalent to 1/10 to the power of x_dat

    if True    :
        x_dat = 1.0/10.0 ** x_dat
        msg = "xdf modified"
    else:
        s_dat = np.log10(1.0 / s_dat)
        msg = "snirf modified"

    # # check that x_dat == s_data  (or almost)
    # isEqual = np.allclose(x_dat, s_dat, atol=0.0001)

    # if isEqual:
    #     print("Data are equal")
    # else:
    #     print("Data are not equal")

    # print the first and last 10 values of x_dat and s_dat
    # n = 5
    # print("first:  x{} ".format(x_dat[:n]))
    # print("        s{} ".format( s_dat[:n]))
    # print("  diff:  x{} ".format(x_dat[:n] - s_dat[:n]))    

    # get the error
    error = x_dat - s_dat
    # get the percentage of error 
    percent_error = 2 * error / (x_dat+s_dat) * 100


    errors.append(error)
    percent_errors.append(percent_error)



    plt.figure()
    plt.plot(x_time, x_dat, "o-", label=x_labels[ch])
    plt.plot(s_time, s_dat, "o-", label=s_labels[ch])
    plt.xlabel("Time (s)")
    plt.ylabel("OD")
    plt.title("xdf + snirf Channel {}".format(ch))
    plt.legend()
    plt.show()


# make a boxplot of the errors
plt.figure()
plt.boxplot(errors)
plt.xticks(range(1, len(x_labels) + 1), x_labels)
# write labels vertically
plt.xticks(rotation=90)
plt.title("Errors (xdf - snirf)")
plt.show()


# make a boxplot of the errors
plt.figure()
plt.boxplot(percent_errors)
plt.xticks(range(1, len(x_labels) + 1), x_labels)
# write labels vertically
plt.xticks(rotation=90)
plt.title("percentage of error (xdf - snirf) ")
plt.show()


# make single global boxplot of the errors
percent_errors = np.array(percent_errors).flatten()
plt.figure()
plt.boxplot(percent_errors)
plt.title("percentage of error (xdf - snirf): "+msg)
plt.show()

# make a histogram of the errors
# get the number of classes in the histogram
n_classes = np.unique(percent_errors).shape[0]
print("n_classes = {}".format(n_classes))
print("numel(percent_errors) = {}".format(len(percent_errors)))

plot_density = True
plt.figure()
plt.hist(percent_errors, bins=100, log=False, density=plot_density)
plt.title("percentage of error (xdf - snirf): "+msg)
if plot_density:
    plt.ylabel("Probability Density")
else:
    plt.ylabel("Count")
plt.show()

# make a single global dot plot of the errors
plt.figure()
plt.plot(percent_errors, "o")
plt.title("percentage of error (xdf - snirf): "+msg)
plt.show()



In [ ]:
# 

In [ ]:
# new figure
plt.figure()

myTime_s = raw.times - raw.annotations.onset[0]
myTime_s_event = annotations.onset - annotations.onset[iStart]

myTime_x = nirsTime_xdf - eventTime_xdf[iStart]
myTime_x_event = eventTime_xdf - eventTime_xdf[iStart]

# subplot 1
plt.subplot(2, 1, 1)

# plot the signal
for i in range(1):  # raw.get_data().shape[0]):
    toPlot = raw.get_data()[i, :]
    # toPlot = np.append(np.diff(toPlot), 0)
    plt.plot(myTime_s, toPlot)

# toPlot_x = nirsData[:, 1]
# toPlot_x = -(toPlot_x - np.mean(toPlot_x)) +  np.mean(toPlot_x)
# toPlot_x = np.append(np.diff(toPlot_x), 0)
# plt.plot(myTime_x, toPlot_x, color="black")

plt.axvline(x=0, color="red")


# plot the anotations
for i in range(len(annotations.description)):
    e = annotations.description[i]
    t = myTime_s_event[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green", linestyle="dashed")
    elif "111" in e:
        plt.axvline(x=t, color="red", linestyle="dashed")


plt.subplot(2, 1, 2)

# plot the signal
for i in range(1):  # , raw.get_data().shape[0]):
    toPlot = raw.get_data()[i, :]
    # toPlot = np.append(np.diff(toPlot), 0)
    plt.plot(myTime_s, toPlot)

# plot the events
for i in range(len(eventData_xdf)):
    e = eventData_xdf[i][0]
    t = myTime_x_event[i]
    plt.axvline(x=t, color="lightgray")
    if "100" in e:
        plt.axvline(x=t, color="green")
    elif "111" in e:
        plt.axvline(x=t, color="red")

plt.show()